In [1]:
import pandas as pd
import utils
import numpy as np
import warnings
from tqdm import tqdm
from copairs.map import average_precision, mean_average_precision
import logging

logging.basicConfig(format="%(levelname)s:%(asctime)s:%(name)s:%(message)s")
logging.getLogger("copairs").setLevel(logging.INFO)

warnings.simplefilter(action="ignore", category=FutureWarning)

c:\Users\ssivagur\Anaconda3\envs\copairs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#operations = "wellpos_cc_var_mad_outlier_featselect_sphering_harmony_PCA_corrected"
batch_size = 20000
null_size = 20000
fdr = 0.1

In [3]:
# CRISPR controls - did not use this since the profies already has pert_type in it

#crispr_controls_df = pd.DataFrame(
#    {
#        "Metadata_JCP2022": ["JCP2022_805264", "JCP2022_800001", "JCP2022_800002"],
#        "Metadata_pert_type": ["poscon", "negcon", "negcon"],
##
#    index=[0, 1, 2],
#)

### Prepare the data

#### Filtering the ORF plates from the output file

In [3]:
raw_CPD_df = pd.read_parquet('C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\CPDPipelineValidation\\profiles_var_mad_int_featselect_harmony.parquet')

In [4]:
raw_CPD_df.shape

(950428, 744)

In [5]:
#filtering 
crispr_df = raw_CPD_df[raw_CPD_df['Metadata_PlateType'] == 'CRISPR']

There are two columns of metadata for 'Metadata_Symbol' in the raw_orf_df - 'Metadata_Symbol_x' and 'Metadata_Symbol_y'. I have to check why there are two columns 

In [6]:
crispr_df.shape

(56821, 744)

In [7]:
crispr_df_cols = [c for c in crispr_df.columns if c.startswith("Metadata_")]

In [8]:
crispr_df_cols

['Metadata_Source',
 'Metadata_Plate',
 'Metadata_Well',
 'Metadata_JCP2022',
 'Metadata_broad_sample',
 'Metadata_Name',
 'Metadata_Vector',
 'Metadata_Transcript',
 'Metadata_Symbol_x',
 'Metadata_NCBI_Gene_ID_x',
 'Metadata_Taxon_ID',
 'Metadata_Gene_Description',
 'Metadata_Prot_Match',
 'Metadata_Insert_Length',
 'Metadata_pert_type',
 'Metadata_NCBI_Gene_ID_y',
 'Metadata_Symbol_y',
 'Metadata_Batch',
 'Metadata_PlateType',
 'Metadata_Row',
 'Metadata_Column',
 'Metadata_Microscope']

In [9]:
crispr_df['Metadata_NCBI_Gene_ID_x']

428536    NaN
428537    NaN
428538    NaN
428539    NaN
428540    NaN
         ... 
487635    NaN
487636    NaN
487637    NaN
487638    NaN
487639    NaN
Name: Metadata_NCBI_Gene_ID_x, Length: 56821, dtype: category
Categories (12602, object): ['1', '10', '100', '10000', ..., 'XLOC_l2_015578', 'XLOC_l2_015600', 'XLOC_l2_015937', 'eGFP']

In [10]:
crispr_df['Metadata_Symbol_x']

428536    NaN
428537    NaN
428538    NaN
428539    NaN
428540    NaN
         ... 
487635    NaN
487636    NaN
487637    NaN
487638    NaN
487639    NaN
Name: Metadata_Symbol_x, Length: 56821, dtype: category
Categories (12602, object): ['A1BG', 'A1CF', 'A2M', 'A4GALT', ..., 'ZYG11B', 'ZYX', 'ZZZ3', 'eGFP']

In [11]:
#dropping the columns ['Metadata_Symbol_x'] and ['Metadata_NCBI_Gene_ID_x'] since they have NaNs
crispr_df = crispr_df.drop(['Metadata_Symbol_x', 'Metadata_NCBI_Gene_ID_x'], axis=1)

In [12]:
#renaming the ['Metadata_Symbol_y'] and ['Metadata_NCBI_Gene_ID_y'] to be used downstream 
crispr_df = crispr_df.rename(columns={'Metadata_NCBI_Gene_ID_y':'Metadata_NCBI_Gene_ID', 'Metadata_Symbol_y':'Metadata_Symbol'})

In [13]:
crispr_df.shape

(56821, 742)

In [14]:
crispr_df.to_parquet('C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\CPDPipelineValidation\\profiles_crispr.parquet')

The size of the original file is (51185, 263)

#### Add annotations - not running this since the profiles alreayd has the metadata in it 

In [11]:
#    pd.read_csv(
#        "../00.download-and-process-annotations/output/crispr_metadata.tsv.gz", sep="\t"
#    )
#   .merge(crispr_controls_df, on="Metadata_JCP2022", how="left")
#    .fillna(value={"Metadata_pert_type": "trt"})
#)
#compound_metadata_df = pd.read_csv(
#    "../datasets/metadata/compound.csv.gz", usecols=["Metadata_JCP2022"]
#).assign(
#    Metadata_pert_type=lambda x: np.where(
#        x["Metadata_JCP2022"] == "JCP2022_999999", "empty", "poscon"
#    )
#)

#metadata_df = pd.concat(
#    [crispr_metdata_df, compound_metadata_df],
#    join="outer",
#    ignore_index=True,
#)

#crispr_df = crispr_df.merge(metadata_df, on="Metadata_JCP2022", how="inner")
#crispr_df.shape

(56821, 651)

#### Remove `poscon` wells.

In [15]:
crispr_df = crispr_df.query('Metadata_pert_type!="poscon"').reset_index(drop=True)
crispr_df.shape

(51744, 742)

#### Remove featues with `nan` values.
These need to be removed as the `nan` values will cause the mean average precision calculation to fail.

In [16]:
crispr_df = utils.remove_nan_features(crispr_df)
crispr_df.shape

Removed nan features: []


(51744, 742)

#### Remove reagents with only one replicate

In [17]:
crispr_df.Metadata_JCP2022.value_counts()[
        crispr_df.Metadata_JCP2022.value_counts() == 1
    ].reset_index()['Metadata_JCP2022'].tolist()

['JCP2022_807435', 'JCP2022_803433', 'JCP2022_807044']

In [18]:
#changed the code to the column name
reagents_with_one_replicate = (
    crispr_df.Metadata_JCP2022.value_counts()[
        crispr_df.Metadata_JCP2022.value_counts() == 1
    ]
    .reset_index()["Metadata_JCP2022"] #.reset_index()["index"]
    .tolist()
)


In [19]:

crispr_df = crispr_df[~crispr_df.Metadata_JCP2022.isin(reagents_with_one_replicate)]
crispr_df.shape

(51741, 742)

### Calculate mAP for each ORF perturbation

In [20]:
# Adding a new column for negative control
crispr_df["Metadata_negcon"] = np.where(crispr_df["Metadata_pert_type"] == "negcon", 1, 0)

In [21]:
pos_sameby = ["Metadata_JCP2022"]
pos_diffby = []
neg_sameby = ["Metadata_Plate"]
neg_diffby = ["Metadata_negcon"]

In [22]:
metadata_df = utils.get_metadata(crispr_df)
feature_df = utils.get_featuredata(crispr_df)
feature_values = feature_df.values

In [23]:
result = average_precision(
    metadata_df, feature_values, pos_sameby, pos_diffby, neg_sameby, neg_diffby, batch_size=batch_size
)

INFO:2024-08-15 19:45:46,358:copairs:Indexing metadata...
INFO:2024-08-15 19:45:46,503:copairs:Finding positive pairs...
INFO:2024-08-15 19:45:57,945:copairs:Finding negative pairs...
INFO:2024-08-15 19:45:59,431:copairs:Computing positive similarities...
INFO:2024-08-15 19:47:04,499:copairs:Computing negative similarities...
INFO:2024-08-15 19:47:10,912:copairs:Building rank lists...
INFO:2024-08-15 19:47:49,583:copairs:Computing average precision...
INFO:2024-08-15 19:47:50,497:copairs:Creating result DataFrame...
INFO:2024-08-15 19:47:50,506:copairs:Finished.


In [24]:
# Remove negcon
result = result.query('Metadata_pert_type!="negcon"').reset_index(drop=True)

In [25]:
agg_result = (
    mean_average_precision(result, pos_sameby, null_size=null_size, threshold=fdr, seed=12527)
    .rename(columns={'average_precision': 'mean_average_precision'})
)

INFO:2024-08-15 19:49:40,266:copairs:Computing null_dist...
INFO:2024-08-15 19:49:40,388:copairs:Computing p-values...


In [26]:
agg_result.to_csv(f"C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\CPDPipelineValidation\\PhenotypicActivity_CRISPR.csv.gz", index=False)